<a href="https://colab.research.google.com/github/abarpan3/HackDay/blob/main/distribute_hack_the_olives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init; git pull https://github.com/abarpan3/HackDay.git

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 21 (delta 4), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (21/21), done.
From https://github.com/abarpan3/HackDay
 * branch            HEAD       -> FETCH_HEAD


# SVM and Ensembles

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from IPython.core.display import Image

## Italian Olives

![](https://github.com/abarpan3/HackDay/blob/main/images/Italy.png?raw=1)

I found this data set in the RGGobi book (http://www.ggobi.org/book/), from which the above diagram is taken. It has "the percentage composition of fatty acids
found in the lipid fraction of Italian olive oils', with oils from 3 regions of Italy: the North, the South, and Sardinia. The regions themselves are subdivided into areas as shown in the map above. The source for this data is:

>Forina, M., Armanino, C., Lanteri, S. & Tiscornia, E. (1983), Classification of Olive Oils from their Fatty Acid Composition, in Martens, H. and
Russwurm Jr., H., eds, Food Research and Data Analysis, Applied Science
Publishers, London, pp. 189–214.

In [ ]:
df=pd.read_csv("data/olives-cleaned.csv")
df.head(5)

### exploring globally

In [ ]:
pd.value_counts(df.areastring, sort=False).plot(kind="bar");

In [ ]:
pd.value_counts(df.regionstring, sort=False).plot(kind="barh");

In [ ]:
df.describe()

## Figuring the South of Italy by Area

In [ ]:
dfsouth=df[df.regionstring=='South']
dfsouth.head()

In [ ]:
dfnosouth=df[df.regionstring!='South']
dfnosouth.head()

## SVM on training data for Regions

Diagram from Jesse Johnson's excellent Shape of Data;
http://shapeofdata.wordpress.com/2013/05/14/linear-separation-and-support-vector-machines/


![max margin](https://github.com/abarpan3/HackDay/blob/main/images/svm.png?raw=1) 

The idea is to draw a line in space between the classes. But not any line, but the line which gives the `maximum margin` rectangle between points of different classes.

from http://nlp.stanford.edu/IR-book/html/htmledition/support-vector-machines-the-linearly-separable-case-1.html :
![support vectors](https://github.com/abarpan3/HackDay/blob/main/images/img1260.png?raw=1)

The points right at the boundary are called the support vectors, which is where the name comes from.

But what if the separability is not so simple, and there are points intruding?

![intrusion](https://github.com/abarpan3/HackDay/blob/main/images/svm21.png?raw=1)

Then the idea is to minimize the distance of the "crossed over" points from the separating line. These crossed over points are costed using "slack" vectors. You dont want too many of these.

You obtain the line by minimizing the [Hinge Loss](http://en.wikipedia.org/wiki/Hinge_loss)

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#Stolen from Jake's notebooks, above: https://github.com/jakevdp/ESAC-stats-2014
from sklearn.datasets import make_blobs
from sklearn.svm import SVC # "Support Vector Classifier"

def plot_svc_decision_function(clf, ax=None):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    x = np.linspace(plt.xlim()[0], plt.xlim()[1], 30)
    y = np.linspace(plt.ylim()[0], plt.ylim()[1], 30)
    Y, X = np.meshgrid(y, x)
    P = np.zeros_like(X)
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            P[i, j] = clf.decision_function([[xi, yj]])
    return ax.contour(X, Y, P, colors='k',
                      levels=[-1, 0, 1], alpha=0.5,
                      linestyles=['--', '-', '--'])

def plot_svm(N):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    clf = SVC(kernel='linear')
    clf.fit(X, y)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='spring')
    plt.xlim(-1, 4)
    plt.ylim(-1, 6)
    plot_svc_decision_function(clf, plt.gca())
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
                s=200, facecolors='none')
    
#interact(plot_svm, N=[10, 200]);
interact(plot_svm, N=widgets.IntSlider(min=10, max=200, step=10, value=90));

Notice how the points that mainly matter are the ones which are near the support vector. If new such points come in, the prediction can change.

## Training a SVM

We just do two acids to start and illustrate. We'll try and see if an Olive oil is from Sardinia or not!

In [ ]:
X = dfnosouth[['linoleic', 'arachidic']]
X = dfnosouth[['linolenic', 'arachidic']]


y = (dfnosouth.regionstring.values=='Sardinia')*1

### Q1: Standardize the data set the correct way

The correct way, unlike what we have done earlier is to standardize the training and test sets separately. Why is this?

You are **not supposed to have seen the test set**. If you do, you are contaminating the training set and your results and error will have an optimistic bias on the training set.

Indeed you will need to standardize the test set data to do your predictions by the mean and the standard deviation learned on the training set!

Store the standardized data in `Xtrain, Xtest, ytrain, ytest`

In [ ]:
# your code here


In [ ]:
Xtrain.shape, Xtest.shape

Now fit the SVM with a "linear" kernel.

In [ ]:
# your code here


In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(Xtrain[:, 0], Xtrain[:, 1], c=ytrain, s=50, cmap='spring', alpha=0.3)
plot_svc_decision_function(clf, plt.gca())
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
                s=100, edgecolors='k', facecolors='none')
plt.scatter(Xtest[:, 0], Xtest[:, 1], c=ytest, s=50, marker="s", cmap='spring', alpha=0.5);

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(Xtrain[:, 0], Xtrain[:, 1], c=ytrain, s=50, cmap='spring', alpha=0.3)
plot_svc_decision_function(clf, plt.gca())
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
                s=100, edgecolors='k', facecolors='none')
plt.scatter(Xtest[:, 0], Xtest[:, 1], c=clf.predict(Xtest), s=50, marker="s", cmap='spring', alpha=0.5);

What is our accuracy?

In [ ]:
# your code here


## The Kernel Trick

Often in SVMs one uses the [kernel trick](http://en.wikipedia.org/wiki/Kernel_method), which maps a lower dimension to a higher one to make things separable.

See (from above mentioned book)

![](https://github.com/abarpan3/HackDay/blob/main/images/img1331.png?raw=1)

So lets see what using a Radial Gaussian kernel look like?

$$e^{-\gamma d(x_1,x_2)^2}$$


### Q2: Train a radial kernel

Train a radial kernel by looking up the documentation. For simplicity we'll start by using the default hyperparameters

In [ ]:
# your code here


In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(Xtrain[:, 0], Xtrain[:, 1], c=ytrain, s=50, cmap='spring', alpha=0.3)
plot_svc_decision_function(clf, plt.gca())
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
                s=100, facecolors='none', edgecolors='k')
plt.scatter(Xtest[:, 0], Xtest[:, 1], c=ytest, s=50, marker="s", cmap='spring', alpha=0.5);

 Compute the confusion matric and the accuracy score!

In [ ]:
# your code here


In [ ]:
clf.score(Xtest, ytest)

### Q3: Use cross validation

This is a take away for home. Use cross validation to estimate the hyper-parameters of this model. These are the regularization and the kernel radius.

This page will help: https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

Discuss on discourse!

## Detective work

See if you can separate out sicilian olive oils using either SVM or logistic regression or kNN.

When you do this you might learn something about how the olive oil industry works...

Discuss on discourse!